In [1]:
!pip install --upgrade gensim

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
import pandas as pd
import os
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import re
import ipywidgets as widgets
import gensim

2022-06-19 14:36:58.565908: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-19 14:36:58.565958: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Datensatz einlesen
df = pd.read_csv('data/out.csv')

In [4]:
# Dataframe mit 3 Spalten. Werden so gejoint, dass ein neues Dataframe mit ein Haiku pro Zeile erstellt wird
df = df[['0', '1', '2']].agg(lambda x: '\n'.join(x.values), axis=1)
# Dataframe to list [[]] -> []
array_of_poems = df.values.tolist()

In [5]:
# num_poems an Haikus mit Semicolon zwshceneinander zusammenfuegen
num_poems = 10000

text = ';'.join(array_of_poems[:num_poems])

In [6]:
# Anzahl der unterschielichen characters im gesamt datensatz herausfinden
vocab = sorted(set(text))
vocab_size = len(vocab)
print(f'{vocab_size} unique chars')
print(vocab)

29 unique chars
['\n', ' ', ';', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [7]:
# Dictionary erstellen. Jeder character wird mit einer Zahl nummeriert
char_indices = {c: i for i, c in enumerate(vocab)}
indices_char = {i: c for i, c in enumerate(vocab)}

# cut the text in semi-redundant sequences of seq_len characters
seq_len = 100

In [8]:
model = keras.models.load_model('myModelTuner.h5')

2022-06-19 14:37:05.547198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-19 14:37:05.547262: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-19 14:37:05.547288: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-acs277): /proc/driver/nvidia/version does not exist
2022-06-19 14:37:05.547578: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model2 = keras.models.load_model('myModelEmbedding.h5')

In [10]:
# Standartfunktion Probability array to onehot to integerencoded char
# [0.3, 0.2, 0.1, 0.4] -> [0, 0, 0, 1] -> return 4 (stelle, an der 1)
def sample(prob, temperature=1.0):
    np.seterr(divide = 'ignore')
    #np.seterr(divide = 'warn') 
    # helper function to sample an index from a probability array
    prob = np.asarray(prob).astype("float64")
    prob = np.log(prob) / temperature
    exp_prob = np.exp(prob)
    prob = exp_prob / np.sum(exp_prob)
    probas = np.random.multinomial(1, prob)
    return np.argmax(probas)

In [11]:

output = widgets.Output()
@output.capture(clear_output=True)
def generate_haiku_characterbased(haiku_start, temperature=1.0):
    lenString = 300
    randomNum = random.randint(0, len(text) - lenString)
    findTemplate = text[randomNum: randomNum + lenString]

    pattern = '(.{99};)'
    seed = re.search(pattern, findTemplate, re.DOTALL).group()

    poem = format(haiku_start.value)
    seed = seed[len(poem):] + poem


    print(poem, end='')
    
    while(len(poem) == 0 or poem[-1] != ';'):
        x_pred = np.zeros((1, len(seed), vocab_size))
        for t, char in enumerate(seed):
            x_pred[0, t, char_indices[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]
        seed = seed[1:] + next_char
        poem += next_char
        print(next_char, end='')



In [12]:
df2 = pd.read_csv('data/out.csv')

In [13]:
# Dataframe mit 3 Spalten. Werden so gejoint, dass ein neues Dataframe mit ein Haiku pro Zeile erstellt wird
df2 = df2[['0', '1', '2']].agg(lambda x: ' \n '.join(x.values), axis=1)
# Dataframe to list [[]] -> []
haikus = df2.values.tolist()

In [14]:
# alle Haikus in Array
#number_of_haikus = len(haikus)
number_of_haikus = 10000

haikus = haikus[:number_of_haikus]


print('number of haikus: ' + str(number_of_haikus))

number of haikus: 10000


In [15]:
# die Haikus cleanen und selber auch noch mal als Wort-Arrays in großen Array
haikus = np.array(haikus)

def clean_and_split(sentence):
    result = list(filter(''.__ne__, re.sub('[.,_]', '', sentence).split(' ')))
    result.append(';')
    return result

haikus = list(map(lambda x: clean_and_split(x), haikus))

In [16]:
word_model = gensim.models.Word2Vec.load('w2v_model.bin')

In [17]:
def word2idx(word):
    return word_model.wv.key_to_index[word]
def idx2word(idx):
    return word_model.wv.index_to_key[idx]

In [18]:
#haikus_combined =  sum(haikus, [])

haikus_combined = [item for sublist in haikus for item in sublist]
# cut the text in semi-redundant sequences of seq_len characters
seq_len = 20
step = 1
# Input String
sequences = []

for i in range(0, len(haikus_combined) - seq_len, step):
    sequences.append(haikus_combined[i : i + seq_len])

train_x = np.zeros([len(sequences), seq_len], dtype=np.int32)



for i, sequence in enumerate(sequences):
    for t, word in enumerate(sequence):
         train_x[i, t] = word2idx(word)



generation = []
for i, sequence in enumerate(train_x):
    if(train_x[i, seq_len- 1] == word2idx(";")):
        generation.append(sequence)
        
print(len(generation))

9998


In [19]:
def sample2(preds, temperature=1.0):
    np.seterr(divide = 'ignore')
    #np.seterr(divide = 'warn') 
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [20]:
def generate_next(text, num_generated=10):
    word_idxs = [word2idx(word) for word in text.lower().split()]
    for i in range(num_generated):
        prediction = model.predict(x=np.array(word_idxs))
        idx = sample(prediction[-1], temperature=0.7)
        word_idxs.append(idx)
    return ' '.join(idx2word(idx) for idx in word_idxs)

In [21]:
output2 = widgets.Output()
@output2.capture(clear_output=True)
def generate_haiku_wordbased(haiku_start, temperature=1.0):
    
    haiku_start = format(haiku_start.value)
    
    split_words = list(filter(''.__ne__, re.sub('[.,_]', '', haiku_start).split(' ')))
    
    start_index = random.randint(0, len(generation)-1)
    generate_with =  generation[start_index]
    
    for word in split_words:
        generate_with = np.append(generate_with[1:], word2idx(word))


    seed = np.zeros([1, seq_len], dtype=np.int32)
    for i in range(seq_len):
        seed[0][i] = generate_with[i]

    pred_word = ""
    print(haiku_start, end=' ')
    while(pred_word != ";"):
        preds = model2.predict(seed)
        pred_index = sample2(preds[-1])
        pred_word = idx2word(pred_index)
        seed_tmp = np.concatenate((seed[0][1:], [pred_index]))
        seed[0] = seed_tmp
        print(pred_word, end=' ')
    
        

In [25]:
input_haiku = widgets.Text(value="",
                           placeholder='type start of haiku',
                           disable=False
)

description = widgets.HTML('Generate characterbased Haiku')

display(description)
display(input_haiku)

input_haiku.on_submit(generate_haiku_characterbased)
display(output)

HTML(value='Generate characterbased Haiku')

Text(value='', placeholder='type start of haiku')

HTML(value='Generate word-based Haiku')

Text(value='i want', placeholder='type start of haiku')

Output(outputs=({'name': 'stdout', 'text': 'ok im always\n san of barkgreat at second\nagends some instince;',…

Output(outputs=({'name': 'stdout', 'text': 'normal baby haley \n was wondering in a friend \n of college accou…

In [ ]:
input_haiku = widgets.Text(value="",
                           placeholder='type start of haiku',
                           disable=False
)

description = widgets.HTML('Generate word-based Haiku')

display(description)
display(input_haiku)